In [1]:
# Import our dependencies
import pandas as pd
from urllib.parse import quote_plus
import numpy as np
from pathlib import Path
%pip install python-dotenv
from dotenv import load_dotenv
import os

# Send output to database
from sqlalchemy import create_engine
# Importing config file for pulling from database
# from config import config

# Load the environment variables from .env file
load_dotenv()

# Accessing the environment variables
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT", 5432)  
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Read a starting database
file_path = "Resources/14100443.csv"
data = pd.read_csv(file_path, on_bad_lines="skip")
data.sample(n=3)

,REF_DATE,GEO,DGUID,National Occupational Classification,Job vacancy characteristics,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
6515546,2016-01,Nova Scotia,2021A000212,Editors [51110],"Minimum level of education required, all levels",Job vacancies,Number,223,units,0,v1570331680,4.542.4.1,NaN,F,NaN,NaN,0
10198630,2016-07,Alberta,2021A000248,Human resources and recruitment officers [12101],5 years to less than 8 years,Job vacancies,Number,223,units,0,v1570962015,10.337.21.1,NaN,F,NaN,NaN,0
14160020,2017-04,Nova Scotia,2021A000212,Technical occupations in life sciences [2211],No professional or other type of certification...,Job vacancies,Number,223,units,0,v1570283434,4.179.16.1,NaN,F,NaN,t,0


<h1><span style="color:green"> I. Removing Redundant Data</span></h1>

### Filter data quality: A - excellent, B - very good, C - good, acceptable - D
### Making Ref Date into actual Date

In [3]:
df = data.copy()
df = df.loc[(df['STATUS'] == 'A')|(df['STATUS'] == 'B')|(df['STATUS'] == 'C')|(df['STATUS'] == 'D')]

#appending date to allow for date time data type
df['REF_DATE'] = df['REF_DATE'] + '-01'

df['REF_DATE'].unique()

array(['2015-01-01', '2015-04-01', '2015-07-01', '2015-10-01',
       '2016-01-01', '2016-04-01', '2016-07-01', '2016-10-01',
       '2017-01-01', '2017-04-01', '2017-07-01', '2017-10-01',
       '2018-01-01', '2018-04-01', '2018-07-01', '2018-10-01',
       '2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01',
       '2020-01-01', '2020-10-01', '2021-01-01', '2021-04-01',
       '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01',
       '2022-07-01', '2022-10-01', '2023-01-01', '2023-04-01',
       '2023-07-01', '2023-10-01', '2024-01-01', '2024-04-01'],
      dtype=object)

## Filtering for Vacancies and Average Wage only

In [4]:
#Filtering for Vacancies and Average Wage
df = df.loc[(df['Statistics'] =='Job vacancies') | (df['Statistics'] == 'Average offered hourly wage')]

In [5]:
#splitting NOC Description and Code
df[['NOCdesc','NOC']] = df['National Occupational Classification'].str.split("[",expand=True)

#removing junk from column
df[['NOC','junk']] = df['NOC'].str.split("]", expand=True)

#dropping columns not required, renaming and reordering columns
df = df.drop(columns=['DGUID','UOM','UOM_ID','SCALAR_FACTOR','SCALAR_ID','VECTOR','STATUS','SYMBOL','COORDINATE',
                 'TERMINATED','DECIMALS','junk','National Occupational Classification'])

df = df.rename(columns={"Job vacancy characteristics" :"JOB_CHAR"})
df = df[['REF_DATE','GEO','NOC','NOCdesc','JOB_CHAR','Statistics','VALUE']]

df.head()

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,Statistics,VALUE
2,2015-01-01,Canada,None,"Total, all occupations","Type of work, all types",Average offered hourly wage,19.1
5,2015-01-01,Canada,None,"Total, all occupations",Full-time,Average offered hourly wage,20.5
8,2015-01-01,Canada,None,"Total, all occupations",Part-time,Average offered hourly wage,15.1
11,2015-01-01,Canada,None,"Total, all occupations","Minimum level of education required, all levels",Average offered hourly wage,19.1
14,2015-01-01,Canada,None,"Total, all occupations",No minimum level of education required,Average offered hourly wage,13.7


In [6]:
#stripping trailing spaces
df["NOCdesc"] = df["NOCdesc"].str.rstrip()

In [7]:
df.isnull().sum()

REF_DATE          0
GEO               0
NOC           38457
NOCdesc           0
JOB_CHAR          0
Statistics        0
VALUE             0
dtype: int64

In [8]:
#checking NOC Descriptions of null NOCs
nullNoc = df[df['NOC'].isna()]
nullNoc['NOCdesc'].unique()

array(['Total, all occupations', 'Unclassified occupations'], dtype=object)

In [9]:
#replacing Null NOCs for Total,all occupations and Unclassified Occupations
temp = df['NOC'].isna()
df.loc[temp, 'NOC'] = np.where(df.loc[temp, 'NOCdesc'].eq('Total, all occupations'), '101', 'X')

In [10]:
#checking outcome
df[df['NOCdesc'] == 'Total, all occupations']
#df[df['NOCdesc'] == 'Unclassified occupations']

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,Statistics,VALUE
2,2015-01-01,Canada,101,"Total, all occupations","Type of work, all types",Average offered hourly wage,19.1
5,2015-01-01,Canada,101,"Total, all occupations",Full-time,Average offered hourly wage,20.5
8,2015-01-01,Canada,101,"Total, all occupations",Part-time,Average offered hourly wage,15.1
11,2015-01-01,Canada,101,"Total, all occupations","Minimum level of education required, all levels",Average offered hourly wage,19.1
14,2015-01-01,Canada,101,"Total, all occupations",No minimum level of education required,Average offered hourly wage,13.7
...,...,...,...,...,...,...,...
55124893,2024-04-01,Nunavut,101,"Total, all occupations",Company website,Job vacancies,280.0
55124895,2024-04-01,Nunavut,101,"Total, all occupations",Online job boards,Job vacancies,215.0
55124897,2024-04-01,Nunavut,101,"Total, all occupations",Social media,Job vacancies,340.0
55124899,2024-04-01,Nunavut,101,"Total, all occupations","Job or recruitment fairs at schools, colleges ...",Job vacancies,130.0



### Creating input table for Machine Learning, filtered on Broad NOC Code, Full/Part Time only

In [11]:
df_vac = df.copy()

# Filter job vacancies
df_vac = df_vac.loc[(df_vac['Statistics'] =='Job vacancies')]

# Filter Job vacancy characteristics
df_vac = df_vac.loc[(df_vac['JOB_CHAR'] == 'Full-time') | (df_vac['JOB_CHAR'] == 'Part-time') | (df_vac['JOB_CHAR'] == 'Type of work, all types')]

#drop column not required
df_vac = df_vac.drop(columns=['Statistics'])

df_vac

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,VALUE
547960,2015-01-01,Quebec,101,"Total, all occupations","Type of work, all types",60480.0
547963,2015-01-01,Quebec,101,"Total, all occupations",Full-time,45455.0
876736,2015-01-01,Saskatchewan,101,"Total, all occupations","Type of work, all types",14330.0
876739,2015-01-01,Saskatchewan,101,"Total, all occupations",Full-time,10420.0
1534288,2015-04-01,Canada,101,"Total, all occupations","Type of work, all types",452315.0
...,...,...,...,...,...,...
55187558,2024-04-01,Nunavut,64,Sales and service representatives and other cu...,Part-time,25.0
55190611,2024-04-01,Nunavut,644,"Occupations in customer, information and prote...","Type of work, all types",30.0
55190617,2024-04-01,Nunavut,644,"Occupations in customer, information and prote...",Part-time,20.0
55195136,2024-04-01,Nunavut,70,Middle management occupations in trades and tr...,Full-time,0.0


In [12]:
print(df_vac.columns)


Index(['REF_DATE', 'GEO', 'NOC', 'NOCdesc', 'JOB_CHAR', 'VALUE'], dtype='object')


In [14]:
# Machine Learning only needs Broad NOC
inc_list = ['101','0','1', '2', '3', '4', '5', '6', '7', '8', '9']

#extract into new dataframe
df_ml = df_vac[df_vac.NOC.isin(inc_list)]

#adding null column to hold predicted vacancies
df_ml["Predicted_Vacancies"] = ''

df_ml

C:\Users\sanja\AppData\Local\Temp\ipykernel_3504\3379917872.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml["Predicted_Vacancies"] = ''


,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,VALUE,Predicted_Vacancies
547960,2015-01-01,Quebec,101,"Total, all occupations","Type of work, all types",60480.0,
547963,2015-01-01,Quebec,101,"Total, all occupations",Full-time,45455.0,
876736,2015-01-01,Saskatchewan,101,"Total, all occupations","Type of work, all types",14330.0,
876739,2015-01-01,Saskatchewan,101,"Total, all occupations",Full-time,10420.0,
1534288,2015-04-01,Canada,101,"Total, all occupations","Type of work, all types",452315.0,
...,...,...,...,...,...,...,...
55124776,2024-04-01,Nunavut,101,"Total, all occupations","Type of work, all types",575.0,
55124779,2024-04-01,Nunavut,101,"Total, all occupations",Full-time,455.0,
55181035,2024-04-01,Nunavut,6,Sales and service occupations,"Type of work, all types",135.0,
55181038,2024-04-01,Nunavut,6,Sales and service occupations,Full-time,95.0,


In [15]:
df_ml=df_ml.rename(columns={"REF_DATE": "ref_date", "GEO": "geo", "JOB_CHAR": "job_char", "NOCdesc": "noc_desc", "NOC": "noc_code", "VALUE":"total_vacancies", "Predicted_Vacancies": "predicted_vacancies"})
df_ml

,ref_date,geo,noc_code,noc_desc,job_char,total_vacancies,predicted_vacancies
547960,2015-01-01,Quebec,101,"Total, all occupations","Type of work, all types",60480.0,
547963,2015-01-01,Quebec,101,"Total, all occupations",Full-time,45455.0,
876736,2015-01-01,Saskatchewan,101,"Total, all occupations","Type of work, all types",14330.0,
876739,2015-01-01,Saskatchewan,101,"Total, all occupations",Full-time,10420.0,
1534288,2015-04-01,Canada,101,"Total, all occupations","Type of work, all types",452315.0,
...,...,...,...,...,...,...,...
55124776,2024-04-01,Nunavut,101,"Total, all occupations","Type of work, all types",575.0,
55124779,2024-04-01,Nunavut,101,"Total, all occupations",Full-time,455.0,
55181035,2024-04-01,Nunavut,6,Sales and service occupations,"Type of work, all types",135.0,
55181038,2024-04-01,Nunavut,6,Sales and service occupations,Full-time,95.0,


In [16]:
df_ml.to_csv('Resources/MachineLearning.csv', index=True, index_label="idML")

In [ ]:
from urllib.parse import quote_plus

# Encode the password to handle special characters
encoded_password = quote_plus(db_password)

# Use the encoded password in the connection string
connection_string = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"

# Create engine and establish connection
engine = create_engine(connection_string)
conn = engine.connect()
df_ml.to_sql(name="machinelearning", con=conn, index=True, index_label="idML", if_exists="replace")
conn.close()


Note: you may need to restart the kernel to use updated packages.


## Creating Vacancies Table data

In [18]:
#adding YEAR and Quarter for Vacancies table

#splitting date to year and month
df_vac[['Year','Quarter','Date']] = df_vac['REF_DATE'].str.split("-",expand=True)                  

#converting Month to Quarter
df_vac['Quarter'] = df_vac['Quarter'].replace(['01','04','07','10'],['1','2','3','4'])

#reordeing columns
df_vac = df_vac[['REF_DATE','Year', 'Quarter', 'GEO','NOC','JOB_CHAR','VALUE']]
df_vac

,REF_DATE,Year,Quarter,GEO,NOC,JOB_CHAR,VALUE
547960,2015-01-01,2015,1,Quebec,101,"Type of work, all types",60480.0
547963,2015-01-01,2015,1,Quebec,101,Full-time,45455.0
876736,2015-01-01,2015,1,Saskatchewan,101,"Type of work, all types",14330.0
876739,2015-01-01,2015,1,Saskatchewan,101,Full-time,10420.0
1534288,2015-04-01,2015,2,Canada,101,"Type of work, all types",452315.0
...,...,...,...,...,...,...,...
55187558,2024-04-01,2024,2,Nunavut,64,Part-time,25.0
55190611,2024-04-01,2024,2,Nunavut,644,"Type of work, all types",30.0
55190617,2024-04-01,2024,2,Nunavut,644,Part-time,20.0
55195136,2024-04-01,2024,2,Nunavut,70,Full-time,0.0


In [19]:
df_vac=df_vac.rename(columns={"REF_DATE": "ref_date", "GEO": "geo", 
                              "JOB_CHAR": "job_char", "NOC": "noc_code", "VALUE":"total_vacancies"})

In [20]:
#exporting to csv
df_vac.to_csv('Resources/AllVacancies.csv', index=True, index_label="idVac")

In [21]:
encoded_password = quote_plus(db_password)

# Use the encoded password in the connection string
connection_string = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"
# Create engine and establish connection
engine2 = create_engine(connection_string)
conn=engine2.connect()
df_vac.to_sql(name="vacancies", con=conn, index=True, index_label="idVac", if_exists="replace")
conn.close()

## NOC Table

In [22]:
## creating NOC dataframe
NOC = df[['NOC','NOCdesc']]

#dropping duplicate rows
NOC = NOC.drop_duplicates()

NOC

,NOC,NOCdesc
2,101,"Total, all occupations"
156,0,Legislative and senior management occupations
289,00,Legislative and senior managers
422,000,Legislative and senior managers
555,0001,Legislative and senior managers
...,...,...
33799825,43200,Sheriffs and bailiffs
36903909,72604,Railway traffic controllers and marine traffic...
43054635,84121,Fishing vessel deckhands
47603628,41310,Police investigators and other investigative o...


In [35]:
from sqlalchemy import text
from urllib.parse import quote_plus

encoded_password = quote_plus(db_password)

# Use the encoded password in the connection string
connection_string = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"

# Create engine
engine3 = create_engine(connection_string)

# Rename columns to match the PostgreSQL table
NOC = NOC.rename(columns={"NOC": "noc_code", "NOCdesc": "noc_desc"})

# Ensure noc_code is a string to match character varying type
NOC['noc_code'] = NOC['noc_code'].astype(str)

# Insert data into the 'noc' table
try:
    with engine3.connect() as conn:
        # Optional: Clear table contents first
        conn.execute(text("DELETE FROM noc;"))
        
        # Insert data
        NOC.to_sql(name="noc", con=engine3, index=False, if_exists="append", method="multi")
        
    print("Data inserted successfully.")
except Exception as e:
    print(f"Error: {e}")

Data inserted successfully.


In [36]:
#exporting to csv
NOC.to_csv('Resources/NOCtable.csv', index=False)

## Creating Data for Average Wage Table

In [28]:
avg_wage = df.copy()

In [29]:
# Filter job vacancies
avg_wage = avg_wage.loc[(avg_wage['Statistics'] =='Average offered hourly wage')]

# Filter number of Job vacancies
avg_wage = avg_wage.loc[(avg_wage['JOB_CHAR'] == 'Full-time') | (avg_wage['JOB_CHAR'] == 'Part-time') | (avg_wage['JOB_CHAR'] == 'Type of work, all types')]

avg_wage

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,Statistics,VALUE
2,2015-01-01,Canada,101,"Total, all occupations","Type of work, all types",Average offered hourly wage,19.10
5,2015-01-01,Canada,101,"Total, all occupations",Full-time,Average offered hourly wage,20.50
8,2015-01-01,Canada,101,"Total, all occupations",Part-time,Average offered hourly wage,15.10
806,2015-01-01,Canada,00011,Senior government managers and officials,Part-time,Average offered hourly wage,12.50
1072,2015-01-01,Canada,00013,"Senior managers - health, education, social an...",Part-time,Average offered hourly wage,42.35
...,...,...,...,...,...,...,...
55202054,2024-04-01,Nunavut,724,Technical maintenance trades,Full-time,Average offered hourly wage,43.55
55202184,2024-04-01,Nunavut,7240,Machinery and transportation equipment mechani...,"Type of work, all types",Average offered hourly wage,43.55
55202187,2024-04-01,Nunavut,7240,Machinery and transportation equipment mechani...,Full-time,Average offered hourly wage,43.55
55202450,2024-04-01,Nunavut,72401,Heavy-duty equipment mechanics,"Type of work, all types",Average offered hourly wage,44.10


In [30]:
#splitting date to year and month
avg_wage[['Year','Quarter','Date']] = avg_wage['REF_DATE'].str.split("-",expand=True)                  

#converting Month to Quarter
avg_wage['Quarter'] = avg_wage['Quarter'].replace(['01','04','07','10'],['1','2','3','4'])

#drop column not required
avg_wage = avg_wage.drop(columns=['Date'])
avg_wage = avg_wage.drop(columns=['Statistics'])

#reordeing columns
avg_wage = avg_wage[['REF_DATE','Year', 'Quarter', 'GEO','NOC','JOB_CHAR','VALUE']]

avg_wage

,REF_DATE,Year,Quarter,GEO,NOC,JOB_CHAR,VALUE
2,2015-01-01,2015,1,Canada,101,"Type of work, all types",19.10
5,2015-01-01,2015,1,Canada,101,Full-time,20.50
8,2015-01-01,2015,1,Canada,101,Part-time,15.10
806,2015-01-01,2015,1,Canada,00011,Part-time,12.50
1072,2015-01-01,2015,1,Canada,00013,Part-time,42.35
...,...,...,...,...,...,...,...
55202054,2024-04-01,2024,2,Nunavut,724,Full-time,43.55
55202184,2024-04-01,2024,2,Nunavut,7240,"Type of work, all types",43.55
55202187,2024-04-01,2024,2,Nunavut,7240,Full-time,43.55
55202450,2024-04-01,2024,2,Nunavut,72401,"Type of work, all types",44.10


In [31]:
avg_wage.drop_duplicates(keep='first', ignore_index=False, inplace=True)
avg_wage

,REF_DATE,Year,Quarter,GEO,NOC,JOB_CHAR,VALUE
2,2015-01-01,2015,1,Canada,101,"Type of work, all types",19.10
5,2015-01-01,2015,1,Canada,101,Full-time,20.50
8,2015-01-01,2015,1,Canada,101,Part-time,15.10
806,2015-01-01,2015,1,Canada,00011,Part-time,12.50
1072,2015-01-01,2015,1,Canada,00013,Part-time,42.35
...,...,...,...,...,...,...,...
55202054,2024-04-01,2024,2,Nunavut,724,Full-time,43.55
55202184,2024-04-01,2024,2,Nunavut,7240,"Type of work, all types",43.55
55202187,2024-04-01,2024,2,Nunavut,7240,Full-time,43.55
55202450,2024-04-01,2024,2,Nunavut,72401,"Type of work, all types",44.10


In [32]:
avg_wage=avg_wage.rename(columns={"REF_DATE": "ref_date", "GEO": "geo", 
                              "JOB_CHAR": "job_char", "NOC": "noc_code", "VALUE":"avg_wage"})
avg_wage

,ref_date,Year,Quarter,geo,noc_code,job_char,avg_wage
2,2015-01-01,2015,1,Canada,101,"Type of work, all types",19.10
5,2015-01-01,2015,1,Canada,101,Full-time,20.50
8,2015-01-01,2015,1,Canada,101,Part-time,15.10
806,2015-01-01,2015,1,Canada,00011,Part-time,12.50
1072,2015-01-01,2015,1,Canada,00013,Part-time,42.35
...,...,...,...,...,...,...,...
55202054,2024-04-01,2024,2,Nunavut,724,Full-time,43.55
55202184,2024-04-01,2024,2,Nunavut,7240,"Type of work, all types",43.55
55202187,2024-04-01,2024,2,Nunavut,7240,Full-time,43.55
55202450,2024-04-01,2024,2,Nunavut,72401,"Type of work, all types",44.10


In [33]:
#exporting to csv
avg_wage.to_csv('Resources/AvgWagetable.csv', index = False)

In [34]:
encoded_password = quote_plus(db_password)

# Use the encoded password in the connection string
connection_string = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"
# Create engine and establish connection
engine4 = create_engine(connection_string)
conn=engine4.connect()
avg_wage.to_sql(name="averagewage", con=conn, index=True, index_label="idW", if_exists="replace")
conn.close()